# Install pyspark library

In [1]:
!pip install pyspark

  Using cached pyspark-3.5.3-py2.py3-none-any.whl
  Using cached py4j-0.10.9.7-py2.py3-none-any.whl.metadata (1.5 kB)
Using cached py4j-0.10.9.7-py2.py3-none-any.whl (200 kB)


In [2]:
#import pyspark library
import pyspark

In [3]:
#import spark session library
from pyspark.sql import SparkSession

In [4]:
# Create SparkSession object
spark = SparkSession.builder \
                    .master('local[*]') \
                    .appName('linear_regression') \
                    .getOrCreate()

# Create DataFrame

In [5]:
#To create dataframe form External datasets
df = spark.read.option("header", "true").csv("../data/*")

In [6]:
df.show()

+----+-------+-----+----------+---------+----------+-------------+---------+-------+-------+---------+---------------+------------------+------------------+------+--------------+-----------+---------------+---------------+---------+-------------+----------------+----------------+----+---------------+---------+-------------+-------------+-------+----------+-------+--------+---------------+--------+--------------------+----------+-------+---------+--------+------+----------+-------+--------+---------------+--------+------------------+----------+---------+----------------+--------+--------------+-----------------+-------+-------+--------+-------------+------------+------------+--------+-------------+-----------------+------------+-------------+---------------+------------------+--------------+--------------------+-----------+-----------+-----------+-------------+----------------+------------+--------------+----------------+-------------+-----------+-----------+-------------+--------------

# Data Preparation

In [7]:
# create new dataframe as per required columns for prediction
AirlineDF = df.select("Origin","Dest","AirTime","Distance")

In [8]:
#cache data in-memory for fast read write operation
AirlineDF.cache()

DataFrame[Origin: string, Dest: string, AirTime: string, Distance: string]

In [9]:
#check data in new dataframe
AirlineDF.show()

+------+----+-------+--------+
|Origin|Dest|AirTime|Distance|
+------+----+-------+--------+
|   JFK| LAX| 338.00| 2475.00|
|   JFK| LAX| 349.00| 2475.00|
|   JFK| LAX| 370.00| 2475.00|
|   JFK| LAX| 350.00| 2475.00|
|   JFK| LAX| 335.00| 2475.00|
|   JFK| LAX| 336.00| 2475.00|
|   JFK| LAX| 380.00| 2475.00|
|   JFK| LAX| 359.00| 2475.00|
|   JFK| LAX| 368.00| 2475.00|
|   JFK| LAX| 356.00| 2475.00|
|   JFK| LAX| 353.00| 2475.00|
|   JFK| LAX| 332.00| 2475.00|
|   JFK| LAX| 339.00| 2475.00|
|   JFK| LAX| 339.00| 2475.00|
|   JFK| LAX| 335.00| 2475.00|
|   JFK| LAX| 340.00| 2475.00|
|   JFK| LAX| 327.00| 2475.00|
|   JFK| LAX| 308.00| 2475.00|
|   JFK| LAX| 315.00| 2475.00|
|   JFK| LAX| 323.00| 2475.00|
+------+----+-------+--------+
only showing top 20 rows



In [10]:
#check data types of each columns
AirlineDF.printSchema()

root
 |-- Origin: string (nullable = true)
 |-- Dest: string (nullable = true)
 |-- AirTime: string (nullable = true)
 |-- Distance: string (nullable = true)



# We need to change data types of columns

In [11]:
#import sql library for data types 
from pyspark.sql.types import IntegerType

In [12]:
AirlineDF = AirlineDF.withColumn("Distance", AirlineDF["Distance"].cast(IntegerType()))

In [13]:
AirlineDF = AirlineDF.withColumn("AirTime", AirlineDF["AirTime"].cast(IntegerType()))

In [14]:
#check data types of each columns
AirlineDF.printSchema()

root
 |-- Origin: string (nullable = true)
 |-- Dest: string (nullable = true)
 |-- AirTime: integer (nullable = true)
 |-- Distance: integer (nullable = true)



# Check is there any null values in DataFrame 

In [15]:
#import libray for sql function col
from pyspark.sql.functions import col

In [16]:
#check null values in each columns
print(AirlineDF.where(col("Origin").isNull()).count())
print(AirlineDF.where(col("Dest").isNull()).count())
print(AirlineDF.where(col("AirTime").isNull()).count())
print(AirlineDF.where(col("Distance").isNull()).count())

0
0
183789
0


# we need to handel null values 

In [17]:
#now drop row as correspondence to null values
AirlineDF = AirlineDF.dropna(subset=["AirTime"])

In [18]:
#check again
print(AirlineDF.where(col("Origin").isNull()).count())
print(AirlineDF.where(col("Dest").isNull()).count())
print(AirlineDF.where(col("AirTime").isNull()).count())
print(AirlineDF.where(col("Distance").isNull()).count())

0
0
0
0


# we need convert distance miles into km

In [19]:
from pyspark.sql.functions import round

# Convert 'mile' to 'km' 
AirlineDF = AirlineDF.withColumn('Distance', round(AirlineDF.Distance * 1.60934, 0))

AirlineDF.show()

+------+----+-------+--------+
|Origin|Dest|AirTime|Distance|
+------+----+-------+--------+
|   JFK| LAX|    338|  3983.0|
|   JFK| LAX|    349|  3983.0|
|   JFK| LAX|    370|  3983.0|
|   JFK| LAX|    350|  3983.0|
|   JFK| LAX|    335|  3983.0|
|   JFK| LAX|    336|  3983.0|
|   JFK| LAX|    380|  3983.0|
|   JFK| LAX|    359|  3983.0|
|   JFK| LAX|    368|  3983.0|
|   JFK| LAX|    356|  3983.0|
|   JFK| LAX|    353|  3983.0|
|   JFK| LAX|    332|  3983.0|
|   JFK| LAX|    339|  3983.0|
|   JFK| LAX|    339|  3983.0|
|   JFK| LAX|    335|  3983.0|
|   JFK| LAX|    340|  3983.0|
|   JFK| LAX|    327|  3983.0|
|   JFK| LAX|    308|  3983.0|
|   JFK| LAX|    315|  3983.0|
|   JFK| LAX|    323|  3983.0|
+------+----+-------+--------+
only showing top 20 rows



In [20]:
AirlineDF.printSchema()

root
 |-- Origin: string (nullable = true)
 |-- Dest: string (nullable = true)
 |-- AirTime: integer (nullable = true)
 |-- Distance: double (nullable = true)



## Verctorize the features

In [22]:
from pyspark.ml.feature import *

  Using cached pandas-2.0.3-cp38-cp38-win_amd64.whl.metadata (18 kB)
  Using cached pytz-2024.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2024.2-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached numpy-1.24.4-cp38-cp38-win_amd64.whl.metadata (5.6 kB)
Using cached pandas-2.0.3-cp38-cp38-win_amd64.whl (10.8 MB)
Using cached numpy-1.24.4-cp38-cp38-win_amd64.whl (14.9 MB)
Using cached pytz-2024.2-py2.py3-none-any.whl (508 kB)
Using cached tzdata-2024.2-py2.py3-none-any.whl (346 kB)


In [23]:
from pyspark.ml.feature import VectorAssembler

In [24]:
vectorizer = VectorAssembler()
vectorizer.setInputCols(["Distance"])
vectorizer.setOutputCol("features")

df_vect = vectorizer.transform(AirlineDF)

In [25]:
df_vect.show()

+------+----+-------+--------+--------+
|Origin|Dest|AirTime|Distance|features|
+------+----+-------+--------+--------+
|   JFK| LAX|    338|  3983.0|[3983.0]|
|   JFK| LAX|    349|  3983.0|[3983.0]|
|   JFK| LAX|    370|  3983.0|[3983.0]|
|   JFK| LAX|    350|  3983.0|[3983.0]|
|   JFK| LAX|    335|  3983.0|[3983.0]|
|   JFK| LAX|    336|  3983.0|[3983.0]|
|   JFK| LAX|    380|  3983.0|[3983.0]|
|   JFK| LAX|    359|  3983.0|[3983.0]|
|   JFK| LAX|    368|  3983.0|[3983.0]|
|   JFK| LAX|    356|  3983.0|[3983.0]|
|   JFK| LAX|    353|  3983.0|[3983.0]|
|   JFK| LAX|    332|  3983.0|[3983.0]|
|   JFK| LAX|    339|  3983.0|[3983.0]|
|   JFK| LAX|    339|  3983.0|[3983.0]|
|   JFK| LAX|    335|  3983.0|[3983.0]|
|   JFK| LAX|    340|  3983.0|[3983.0]|
|   JFK| LAX|    327|  3983.0|[3983.0]|
|   JFK| LAX|    308|  3983.0|[3983.0]|
|   JFK| LAX|    315|  3983.0|[3983.0]|
|   JFK| LAX|    323|  3983.0|[3983.0]|
+------+----+-------+--------+--------+
only showing top 20 rows



In [26]:
print(vectorizer.explainParams())

handleInvalid: How to handle invalid data (NULL and NaN values). Options are 'skip' (filter out rows with invalid data), 'error' (throw an error), or 'keep' (return relevant number of NaN in the output). Column lengths are taken from the size of ML Attribute Group, which can be set using `VectorSizeHint` in a pipeline before `VectorAssembler`. Column lengths can also be inferred from first rows of the data since it is safe to do so but only in case of 'error' or 'skip'). (default: error)
inputCols: input column names. (current: ['Distance'])
outputCol: output column name. (default: VectorAssembler_d76b46fa970f__output, current: features)


# Train Test data Splitting

In [27]:
flights_train, flights_test = df_vect.randomSplit([0.8, 0.2])

In [28]:
flights_train.show()

+------+----+-------+--------+--------+
|Origin|Dest|AirTime|Distance|features|
+------+----+-------+--------+--------+
|   ABQ| ATL|    134|  2042.0|[2042.0]|
|   ABQ| ATL|    136|  2042.0|[2042.0]|
|   ABQ| ATL|    137|  2042.0|[2042.0]|
|   ABQ| ATL|    137|  2042.0|[2042.0]|
|   ABQ| ATL|    138|  2042.0|[2042.0]|
|   ABQ| ATL|    138|  2042.0|[2042.0]|
|   ABQ| ATL|    139|  2042.0|[2042.0]|
|   ABQ| ATL|    139|  2042.0|[2042.0]|
|   ABQ| ATL|    139|  2042.0|[2042.0]|
|   ABQ| ATL|    139|  2042.0|[2042.0]|
|   ABQ| ATL|    140|  2042.0|[2042.0]|
|   ABQ| ATL|    140|  2042.0|[2042.0]|
|   ABQ| ATL|    140|  2042.0|[2042.0]|
|   ABQ| ATL|    140|  2042.0|[2042.0]|
|   ABQ| ATL|    141|  2042.0|[2042.0]|
|   ABQ| ATL|    142|  2042.0|[2042.0]|
|   ABQ| ATL|    142|  2042.0|[2042.0]|
|   ABQ| ATL|    142|  2042.0|[2042.0]|
|   ABQ| ATL|    144|  2042.0|[2042.0]|
|   ABQ| ATL|    144|  2042.0|[2042.0]|
+------+----+-------+--------+--------+
only showing top 20 rows



# Regression Model Training

In [29]:
from pyspark.ml.regression import LinearRegression

In [30]:
lr = LinearRegression()
print(lr.explainParams())

aggregationDepth: suggested depth for treeAggregate (>= 2). (default: 2)
elasticNetParam: the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty. (default: 0.0)
epsilon: The shape parameter to control the amount of robustness. Must be > 1.0. Only valid when loss is huber (default: 1.35)
featuresCol: features column name. (default: features)
fitIntercept: whether to fit an intercept term. (default: True)
labelCol: label column name. (default: label)
loss: The loss function to be optimized. Supported options: squaredError, huber. (default: squaredError)
maxBlockSizeInMB: maximum memory in MB for stacking input data into blocks. Data is stacked within partitions. If more than remaining data size in a partition then it is adjusted to the data size. Default 0.0 represents choosing optimal value, depends on specific algorithm. Must be >= 0. (default: 0.0)
maxIter: max number of iterations (>= 0). (default: 100)
predic

In [31]:
lr.setLabelCol("AirTime")
lr.setFeaturesCol("features")
model = lr.fit(flights_train)

In [32]:
type(model)

pyspark.ml.regression.LinearRegressionModel

# View model summary

In [33]:
print("R2:", model.summary.r2)
print("Intercept: ", model.intercept, "Coefficients", model.coefficients)

R2: 0.9582701022738098
Intercept:  18.170328792752535 Coefficients [0.07330916410185756]


# Model Testing 

In [34]:
df_pred = model.transform(flights_test)
df_pred.show()

+------+----+-------+--------+--------+------------------+
|Origin|Dest|AirTime|Distance|features|        prediction|
+------+----+-------+--------+--------+------------------+
|   ABQ| ATL|    139|  2042.0|[2042.0]|167.86764188874568|
|   ABQ| ATL|    140|  2042.0|[2042.0]|167.86764188874568|
|   ABQ| ATL|    140|  2042.0|[2042.0]|167.86764188874568|
|   ABQ| ATL|    146|  2042.0|[2042.0]|167.86764188874568|
|   ABQ| ATL|    146|  2042.0|[2042.0]|167.86764188874568|
|   ABQ| ATL|    147|  2042.0|[2042.0]|167.86764188874568|
|   ABQ| ATL|    151|  2042.0|[2042.0]|167.86764188874568|
|   ABQ| ATL|    156|  2042.0|[2042.0]|167.86764188874568|
|   ABQ| CVG|    134|  1996.0|[1996.0]|164.49542034006024|
|   ABQ| CVG|    135|  1996.0|[1996.0]|164.49542034006024|
|   ABQ| CVG|    137|  1996.0|[1996.0]|164.49542034006024|
|   ABQ| CVG|    138|  1996.0|[1996.0]|164.49542034006024|
|   ABQ| CVG|    144|  1996.0|[1996.0]|164.49542034006024|
|   ABQ| CVG|    145|  1996.0|[1996.0]|164.4954203400602